In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

In [3]:
import os
from huggingface_hub import login
from google.colab import userdata

# Accessing the secret key from Colab secrets
hf_token = userdata.get("HF_API_KEY")

if hf_token:
    login(token=hf_token)
else:
    raise ValueError("Hugging Face token not found in Colab secrets 'Colab_HF_token'")

In [4]:
import torch
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os

**Load and Preprocess Training Data**

In [6]:
test_df = pd.read_csv('/content/drive/MyDrive/test (6).csv')
test_df.head()

,unique_id,time,x,y,red,nir,swir16,swir22,blue,green,rededge1,rededge2,rededge3,nir08
0,ID_01FHV4,2018-01-03 10:59:22.851,-296455.0,846395.0,0.2920,0.3686,0.4173,0.3869,0.2488,0.2708,0.3211,0.3555,0.3752,0.3862
1,ID_01FHV4,2018-01-03 10:59:22.851,-296455.0,846395.0,0.2920,0.3686,0.4173,0.3869,0.2488,0.2708,0.3211,0.3555,0.3752,0.3862
2,ID_01FHV4,2018-02-12 10:59:25.232,-296455.0,846395.0,0.3510,0.3426,0.4817,0.4577,0.2538,0.2914,0.3684,0.3484,0.3588,0.3628
3,ID_01FHV4,2018-02-12 10:59:25.232,-296455.0,846395.0,0.3510,0.3426,0.4817,0.4577,0.2538,0.2914,0.3684,0.3484,0.3588,0.3628
4,ID_01FHV4,2018-03-14 10:59:24.436,-296455.0,846395.0,0.5312,0.6296,0.6643,0.5882,0.5244,0.5308,0.6016,0.6217,0.6401,0.6404


In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sentinel2_CropSamples_Reorganized.csv')
train_df.head()


,blue,crop_type,green,nir,nir08,red,rededge1,rededge2,rededge3,swir16,swir22,time,unique_id,x,y
0,559,cocoa,771,2404,2585,846,1240,2006,2326,2630,1684,2022-01-03,PIXEL_0001,0.957526,6.899852
1,537,cocoa,916,2448,2750,1096,1464,2124,2390,2851,1823,2022-01-08,PIXEL_0001,0.957526,6.899852
2,540,cocoa,877,2402,2498,1084,1415,1943,2184,3069,2100,2022-01-23,PIXEL_0001,0.957526,6.899852
3,1944,cocoa,2094,3076,3307,2260,2537,2809,3043,3887,2979,2022-01-28,PIXEL_0001,0.957526,6.899852
4,2062,cocoa,2092,2484,2599,2210,2293,2371,2529,2837,2275,2022-02-12,PIXEL_0001,0.957526,6.899852


In [7]:
train_df['crop_type'].value_counts()

,count
crop_type,
cocoa,6359
rubber,6283
oil,5800


In [8]:
from transformers import AutoModel, AutoConfig

model_name = "AminiTech/amini-28M-v1"

# Load model config and model (for PatchTST, not a language model!)
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, config=config, trust_remote_code=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/75.8M [00:00<?, ?B/s]

In [9]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PatchTSTModel(
  (scaler): PatchTSTScaler(
    (scaler): PatchTSTMeanScaler()
  )
  (patchifier): PatchTSTPatchify()
  (masking): PatchTSTMasking()
  (encoder): PatchTSTEncoder(
    (embedder): PatchTSTEmbedding(
      (input_embedding): Linear(in_features=12, out_features=512, bias=True)
    )
    (positional_encoder): PatchTSTPositionalEncoding(
      (positional_dropout): Identity()
    )
    (layers): ModuleList(
      (0-5): 6 x PatchTSTEncoderLayer(
        (self_attn): PatchTSTAttention(
          (k_proj): Linear(in_features=512, out_features=512, bias=True)
          (v_proj): Linear(in_features=512, out_features=512, bias=True)
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (dropout_path1): Identity()
        (norm_sublayer1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout_path2): Identity()
        (norm_sublayer2): LayerNorm((512,), 

In [10]:
import numpy as np

def mean_aggregator(cls_embeddings):
    """
    Fully averages all CLS embeddings, no splitting.

    Args:
        cls_embeddings: list or array of CLS tokens [n, 512]
    Returns:
        final_embedding: mean [512]-dim vector
    """
    cls_embeddings = np.array(cls_embeddings)
    return np.mean(cls_embeddings, axis=0)


In [11]:
def generate_embeddings_sliding_window(df, model, max_length=48, stride=24):
    grouped = df.groupby('unique_id')
    all_embeddings = {}

    for unique_id, group in grouped:
        group = group.sort_values('time')
        features = group[['blue', 'green', 'nir', 'nir08', 'red', 'rededge1',
                          'rededge2', 'rededge3', 'swir16', 'swir22']].values
        sequence_length = len(features)
        print(f"[{unique_id}] Raw sequence length: {sequence_length}")

        # Skip very short sequences
        if sequence_length < max_length:
            print(f"[{unique_id}] Skipped (too short)")
            continue

        # Normalize sequence
        features = (features - features.mean(axis=0)) / (features.std(axis=0) + 1e-6)

        cls_list = []

        for i in range(0, sequence_length - max_length + 1, stride):
            window = features[i : i + max_length]  # shape: [max_length, 10]
            tensor_input = torch.tensor(window, dtype=torch.float32).unsqueeze(0).to(device)  # shape: [1, 48, 10]

            with torch.no_grad():
                try:
                    output = model(past_values=tensor_input)
                    cls_token = output.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()  # shape: [512]
                    cls_list.append(cls_token)
                except Exception as e:
                    print(f"[{unique_id}] Error at window {i}: {e}")
                    continue

        if cls_list:
            all_embeddings[unique_id] = cls_list  # list of [512] vectors
        else:
            print(f"[{unique_id}] No CLS tokens generated")

    return all_embeddings


In [12]:
cls_embeddings = generate_embeddings_sliding_window(train_df, model, max_length=48, stride=24)

[PIXEL_0001] Raw sequence length: 54
[PIXEL_0002] Raw sequence length: 108
[PIXEL_0003] Raw sequence length: 112
[PIXEL_0004] Raw sequence length: 55
[PIXEL_0005] Raw sequence length: 46
[PIXEL_0005] Skipped (too short)
[PIXEL_0006] Raw sequence length: 110
[PIXEL_0007] Raw sequence length: 54
[PIXEL_0008] Raw sequence length: 109
[PIXEL_0009] Raw sequence length: 51
[PIXEL_0010] Raw sequence length: 51
[PIXEL_0011] Raw sequence length: 57
[PIXEL_0012] Raw sequence length: 43
[PIXEL_0012] Skipped (too short)
[PIXEL_0013] Raw sequence length: 55
[PIXEL_0014] Raw sequence length: 55
[PIXEL_0015] Raw sequence length: 56
[PIXEL_0016] Raw sequence length: 54
[PIXEL_0017] Raw sequence length: 46
[PIXEL_0017] Skipped (too short)
[PIXEL_0018] Raw sequence length: 44
[PIXEL_0018] Skipped (too short)
[PIXEL_0019] Raw sequence length: 56
[PIXEL_0020] Raw sequence length: 49
[PIXEL_0021] Raw sequence length: 46
[PIXEL_0021] Skipped (too short)
[PIXEL_0022] Raw sequence length: 51
[PIXEL_0023] Raw 

In [13]:
cls_embeddings_test = generate_embeddings_sliding_window(test_df, model, max_length=48, stride=24)

Streaming output truncated to the last 5000 lines.
[ID_IU4BO4] Raw sequence length: 145
[ID_IU5UN3] Raw sequence length: 80
[ID_IUD2FV] Raw sequence length: 154
[ID_IUI2ZC] Raw sequence length: 74
[ID_IUMD7K] Raw sequence length: 145
[ID_IUPPLK] Raw sequence length: 78
[ID_IV7F4V] Raw sequence length: 145
[ID_IV7K3B] Raw sequence length: 146
[ID_IVB5RH] Raw sequence length: 99
[ID_IVHX9V] Raw sequence length: 129
[ID_IVOZTZ] Raw sequence length: 79
[ID_IVSS65] Raw sequence length: 80
[ID_IVXV4G] Raw sequence length: 74
[ID_IVY1OR] Raw sequence length: 79
[ID_IW4FCT] Raw sequence length: 80
[ID_IW5CQ4] Raw sequence length: 79
[ID_IWLDYH] Raw sequence length: 80
[ID_IWM7HG] Raw sequence length: 80
[ID_IWS1ML] Raw sequence length: 79
[ID_IWWAUI] Raw sequence length: 146
[ID_IX5FB1] Raw sequence length: 129
[ID_IX9GAN] Raw sequence length: 80
[ID_IX9NKY] Raw sequence length: 80
[ID_IXNTOZ] Raw sequence length: 80
[ID_IXUUP4] Raw sequence length: 154
[ID_IXX00U] Raw sequence length: 145
[ID

In [14]:
final_train_embeddings = []
labels = []

for uid, cls_list in cls_embeddings.items():
    agg_embedding = mean_aggregator(cls_list)  # ✅ Full mean pooling, no splitting
    final_train_embeddings.append(agg_embedding)

    crop_type = train_df.loc[train_df['unique_id'] == uid, 'crop_type'].iloc[0]
    labels.append(crop_type)


In [15]:
cls_embeddings_test = generate_embeddings_sliding_window(test_df, model, max_length=48, stride=24)

final_test_embeddings = []
test_uids = []

for uid, cls_list in cls_embeddings_test.items():
    agg_embedding = mean_aggregator(cls_list)  # ✅ Full mean over all CLS tokens
    final_test_embeddings.append(agg_embedding)
    test_uids.append(uid)  # Optional: useful for mapping back later


Streaming output truncated to the last 5000 lines.
[ID_IU4BO4] Raw sequence length: 145
[ID_IU5UN3] Raw sequence length: 80
[ID_IUD2FV] Raw sequence length: 154
[ID_IUI2ZC] Raw sequence length: 74
[ID_IUMD7K] Raw sequence length: 145
[ID_IUPPLK] Raw sequence length: 78
[ID_IV7F4V] Raw sequence length: 145
[ID_IV7K3B] Raw sequence length: 146
[ID_IVB5RH] Raw sequence length: 99
[ID_IVHX9V] Raw sequence length: 129
[ID_IVOZTZ] Raw sequence length: 79
[ID_IVSS65] Raw sequence length: 80
[ID_IVXV4G] Raw sequence length: 74
[ID_IVY1OR] Raw sequence length: 79
[ID_IW4FCT] Raw sequence length: 80
[ID_IW5CQ4] Raw sequence length: 79
[ID_IWLDYH] Raw sequence length: 80
[ID_IWM7HG] Raw sequence length: 80
[ID_IWS1ML] Raw sequence length: 79
[ID_IWWAUI] Raw sequence length: 146
[ID_IX5FB1] Raw sequence length: 129
[ID_IX9GAN] Raw sequence length: 80
[ID_IX9NKY] Raw sequence length: 80
[ID_IXNTOZ] Raw sequence length: 80
[ID_IXUUP4] Raw sequence length: 154
[ID_IXX00U] Raw sequence length: 145
[ID

In [16]:
import numpy as np

# Updated label map with correct crop type names
label_map = {'cocoa': 0, 'rubber': 1, 'oil': 2}  # 'oil' instead of 'oil_palm'

# Extract the first crop_type per unique_id
pixel_labels = train_df.groupby('unique_id').first()['crop_type']

# Map crop types to integer labels
labels = pixel_labels.map(label_map)

# Drop any unmapped values (NaN) and convert to integer array
labels = labels.dropna().astype(int).values

# Output shape and distribution
print("Labels shape:", labels.shape)
print("Label distribution:", np.bincount(labels))


Labels shape: (300,)
Label distribution: [100 100 100]


In [17]:
import numpy as np

X_train = np.array(final_train_embeddings)  # shape: [num_pixels, 512]
y_train = np.array(labels)  # shape: [num_pixels]


In [18]:
print(X_train.shape, y_train.shape)

(251, 5, 512) (300,)


In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Optional: get label → crop name mapping
class_names = le.classes_


In [20]:
X_train = np.array(final_train_embeddings)

# Flatten from shape [N, 1, 512] → [N, 512] if needed
if X_train.ndim == 3 and X_train.shape[1] == 1:
    X_train = X_train.squeeze(1)


In [21]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
clf.fit(X_train, y_train_encoded)



ValueError: Found array with dim 3. LogisticRegression expected <= 2.

In [ ]:
# === Step 3: Define CNN Classifier ===
class TemporalCNN(nn.Module):
    def __init__(self, input_dim, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 64, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        # Input already [B, C, T]
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool(x)
        return self.fc(x)


In [ ]:
# === Step 4: Train the CNN ===
labels_array = np.array(labels)
X_train, X_val, y_train, y_val = train_test_split(X_pca, labels_array, test_size=0.2, stratify=labels_array)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

model = TemporalCNN(input_dim=X_pca.shape[2]).to('cuda' if torch.cuda.is_available() else 'cpu')
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
device = next(model.parameters()).device

for epoch in range(10):
    model.train()
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        xb = xb.permute(0, 2, 1)  # Add this line
        optimizer.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()


In [ ]:
X_test_seq = []
for uid in sorted(test_embeddings.keys()):
    emb = test_embeddings[uid]  # [10, 5, 512]
    emb = np.mean(emb, axis=1)  # [10, 512]
    X_test_seq.append(emb)
X_test_seq = np.stack(X_test_seq)  # [N, 10, 512]

# Flatten for scaling and PCA
X_test_flat = X_test_seq.reshape(-1, 512)
X_test_scaled = scaler.transform(X_test_flat)

# Apply PCA and reshape for CNN
X_test_pca_flat = pca.transform(X_test_scaled)  # [N*10, n_components]
num_patches = X_test_seq.shape[1]               # should be 10
num_pca_dims = pca.n_components_

# Reshape to [N, num_patches, pca_dims] and transpose to [N, pca_dims, num_patches]
X_test_pca = X_test_pca_flat.reshape(len(X_test_seq), num_patches, num_pca_dims)
X_test_pca = X_test_pca.transpose(0, 2, 1)  # [N, D, T] for CNN

# Torch inference
X_test_tensor = torch.tensor(X_test_pca, dtype=torch.float32).to(device)
model.eval()
with torch.no_grad():
    test_probs = torch.softmax(model(X_test_tensor), dim=1).cpu().numpy()


In [ ]:
# === Step 6: Save to Submission Format ===
submission_df = pd.DataFrame(test_probs, columns=['cocoa', 'rubber', 'oil'])
submission_df.insert(0, 'unique_id', sorted(test_embeddings.keys()))
submission_df.to_csv('submission.csv', index=False)
print("✅ Submission saved as 'submission.csv'")


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Reduce dimensions of training embeddings for visualization
# Flatten the training embeddings
X_train_flat = np.stack([
    np.mean(emb, axis=(0, 1))  # [10, 5, 512] → [512]
    for emb in embeddings.values()
]) # [300, 512]

# Apply PCA to the flattened training embeddings for visualization
pca_viz = PCA(n_components=2)
X_train_pca_viz = pca_viz.fit_transform(X_train_flat) # [300, 2]

# Create a DataFrame for plotting training embeddings
train_viz_df = pd.DataFrame(X_train_pca_viz, columns=['PCA1', 'PCA2'])
train_viz_df['crop_type'] = pixel_labels.values # Use the original labels

# Reduce dimensions of test embeddings for visualization
# Flatten the test embeddings
X_test_flat = np.stack([
    np.mean(emb, axis=(0, 1))  # [10, 5, 512] → [512]
    for emb in test_embeddings.values()
]) # [N, 512]

# Apply the same PCA transformation to the flattened test embeddings
X_test_pca_viz = pca_viz.transform(X_test_flat) # [N, 2]

# Create a DataFrame for plotting test embeddings
test_viz_df = pd.DataFrame(X_test_pca_viz, columns=['PCA1', 'PCA2'])
test_viz_df['unique_id'] = sorted(test_embeddings.keys())

# Plotting training embeddings
plt.figure(figsize=(10, 7))
sns.scatterplot(data=train_viz_df, x='PCA1', y='PCA2', hue='crop_type', palette='viridis')
plt.title('PCA of Training Embeddings (2 Components)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

# Plotting test embeddings
plt.figure(figsize=(10, 7))
sns.scatterplot(data=test_viz_df, x='PCA1', y='PCA2', alpha=0.5) # No hue for test as we don't have labels
plt.title('PCA of Test Embeddings (2 Components)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()